In [7]:
# Let's create a Python script that will generate the docker-compose.yml configuration.
# This script will automate the creation of services for multiple server instances.

# Define the number of server instances and the starting port number
num_servers = 6
start_port = 50051

# The base configuration for docker-compose.yml
base_config = {
    'version': '3.8',
    'services': {
        # Placeholder for server configurations
    },
    'networks': {
        'mynetwork': {}
    }
}

# The template for server services
server_service_template = {
    'build': {
        'context': '.',
        'dockerfile': './server/Dockerfile'
    },
    'networks': ['mynetwork'],
    'command': ['bash -c "go run *.go"'],
    'tty': ['true']
}

# The template for the client service
client_service_template = {
    'build': {
        'context': '.',
        'dockerfile': './client/Dockerfile'
    },
    'ports': ['50057:50057'],
    'depends_on': [],
    'networks': ['mynetwork'],
    'environment': ['ADDR=server1:50051']
}

# Generate the server service configurations
for i in range(num_servers):
    service_name = f'server{i+1}'
    service_config = server_service_template.copy()
    service_config['ports'] = [f'{start_port+i}:{start_port+i}']
    base_config['services'][service_name] = service_config.copy()
    client_service_template['depends_on'].append(service_name)

# Add the client service configuration
base_config['services']['client'] = client_service_template

# Using PyYAML to output the generated configuration as YAML
import yaml

# Generate the docker-compose.yml content
docker_compose_content = yaml.dump(base_config, sort_keys=False)

# Write the content to a file
with open('docker-compose-new.yml', 'w') as file:
    file.write(docker_compose_content)

# Return the path of the created file
docker_compose_content


'version: \'3.8\'\nservices:\n  server1:\n    build: &id001\n      context: .\n      dockerfile: ./server/Dockerfile\n    networks: &id002\n    - mynetwork\n    command: &id003\n    - bash -c "go run *.go"\n    tty: &id004\n    - \'true\'\n    ports:\n    - 50051:50051\n  server2:\n    build: *id001\n    networks: *id002\n    command: *id003\n    tty: *id004\n    ports:\n    - 50052:50052\n  server3:\n    build: *id001\n    networks: *id002\n    command: *id003\n    tty: *id004\n    ports:\n    - 50053:50053\n  server4:\n    build: *id001\n    networks: *id002\n    command: *id003\n    tty: *id004\n    ports:\n    - 50054:50054\n  server5:\n    build: *id001\n    networks: *id002\n    command: *id003\n    tty: *id004\n    ports:\n    - 50055:50055\n  server6:\n    build: *id001\n    networks: *id002\n    command: *id003\n    tty: *id004\n    ports:\n    - 50056:50056\n  client:\n    build:\n      context: .\n      dockerfile: ./client/Dockerfile\n    ports:\n    - 50057:50057\n    depe

In [11]:
time_a = 26.848751749 + 26.815037541 + 27.240891707 + 26.264069748 + 26.840733123 + 25.567603054 + 24.523274386 + 24.221233594 + 23.795844428 + 23.237579678
time_b = 21.896540094 + 21.278390135 + 21.527535844 + 22.308705053 + 22.428326052 + 22.810937636 + 23.072766845 + 22.483376094 + 22.451769303 + 21.076394343
time_c = 21.132052829 + 19.824435412 + 20.102671162 + 20.826016287 + 21.076087579 + 21.346899579 + 21.721502413 + 21.904638121 + 21.849128705 + 21.570546621
time_d = 33.310473043 + 33.099696251 + 33.162330293 + 33.786237043 + 33.683926794 + 33.388208293 + 32.842960002 + 33.415183251 + 32.655563460 + 33.090339168
time_e = 31.015088375 + 31.219155543 + 30.659906667 + 30.893880750 + 29.813535806 + 29.941501000 + 31.503222861 + 31.756317028 + 32.188204404 + 31.002243000
time_f = .271693125 + 19.679132787 + 19.912782954 + 20.831653662 + 20.495062954 + 19.898449579 + 19.640146912 + 19.569755245 + 20.124090454 + 20.508092328
print(time_c/10)

21.4071049378
